# Chapter 05 실습 1: CIFAR-10 분류기

## 목표
CIFAR-10 데이터셋으로 CNN 분류기를 구현하고 성능을 분석한다.

In [ ]:
# 필수 라이브러리 임포트
import sys
sys.path.append('..')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성 시드 고정
tf.random.set_seed(42)
np.random.seed(42)

print(f'TensorFlow 버전: {tf.__version__}')
print(f'NumPy 버전: {np.__version__}')

In [ ]:
# CIFAR-10 데이터 로드 및 전처리

# CIFAR-10 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 레이블 배열 압축: (50000, 1) → (50000,)
y_train = y_train.squeeze()
y_test  = y_test.squeeze()

# 픽셀값 정규화: [0, 255] → [0.0, 1.0]
x_train = x_train / 255.0
x_test  = x_test  / 255.0

# 클래스 이름 정의
CLASS_NAMES = ['비행기', '자동차', '새', '고양이', '사슴',
               '개', '개구리', '말', '배', '트럭']
NUM_CLASSES = len(CLASS_NAMES)

print(f'학습 데이터: {x_train.shape}, dtype: {x_train.dtype}')
print(f'테스트 데이터: {x_test.shape}, dtype: {x_test.dtype}')
print(f'학습 레이블: {y_train.shape}, 범위: [{y_train.min()}, {y_train.max()}]')
print(f'클래스 수: {NUM_CLASSES}')
print()

# 클래스별 샘플 수 확인
print('클래스별 학습 샘플 수:')
for i, name in enumerate(CLASS_NAMES):
    count = np.sum(y_train == i)
    print(f'  {i}: {name:<8} → {count:,}개')

# 샘플 이미지 시각화
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for i, ax in enumerate(axes.flatten()):
    # 각 클래스에서 대표 이미지 선택
    idx = np.where(y_train == i)[0][0]
    ax.imshow(x_train[idx])
    ax.set_title(f'{i}: {CLASS_NAMES[i]}')
    ax.axis('off')

plt.suptitle('CIFAR-10 클래스별 샘플 이미지', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# CNN 모델 구성
# 구조: Conv→BN→MaxPool → Conv→BN→MaxPool → Flatten → Dense → Dropout → Dense(출력)

def build_cnn_model(input_shape=(32, 32, 3), num_classes=10):
    """
    CIFAR-10용 CNN 모델 구성
    
    아키텍처:
      - 합성곱 블록 1: Conv2D(32) → BatchNorm → ReLU → MaxPool
      - 합성곱 블록 2: Conv2D(64) → BatchNorm → ReLU → MaxPool
      - 합성곱 블록 3: Conv2D(128) → BatchNorm → ReLU → MaxPool
      - 분류 헤드:    Flatten → Dense(128) → Dropout(0.5) → Dense(10, softmax)
    """
    model = tf.keras.Sequential([
        # ====== 합성곱 블록 1 ======
        # padding='same': 출력 크기 유지 (32x32 → 32x32)
        tf.keras.layers.Conv2D(
            32, (3, 3), padding='same', use_bias=False,
            input_shape=input_shape, name='conv1'
        ),
        tf.keras.layers.BatchNormalization(name='bn1'),   # 배치 정규화: 학습 안정화
        tf.keras.layers.Activation('relu', name='relu1'),
        tf.keras.layers.MaxPooling2D((2, 2), name='pool1'),  # 32x32 → 16x16
        
        # ====== 합성곱 블록 2 ======
        tf.keras.layers.Conv2D(
            64, (3, 3), padding='same', use_bias=False, name='conv2'
        ),
        tf.keras.layers.BatchNormalization(name='bn2'),
        tf.keras.layers.Activation('relu', name='relu2'),
        tf.keras.layers.MaxPooling2D((2, 2), name='pool2'),  # 16x16 → 8x8
        
        # ====== 합성곱 블록 3 ======
        tf.keras.layers.Conv2D(
            128, (3, 3), padding='same', use_bias=False, name='conv3'
        ),
        tf.keras.layers.BatchNormalization(name='bn3'),
        tf.keras.layers.Activation('relu', name='relu3'),
        tf.keras.layers.MaxPooling2D((2, 2), name='pool3'),  # 8x8 → 4x4
        
        # ====== 분류 헤드 ======
        tf.keras.layers.Flatten(name='flatten'),              # (4, 4, 128) → (2048,)
        tf.keras.layers.Dense(128, activation='relu', name='dense1'),
        tf.keras.layers.Dropout(0.5, name='dropout'),         # 과적합 방지
        tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
    ], name='CIFAR10_CNN')
    
    return model


# 모델 생성 및 구조 출력
model = build_cnn_model(input_shape=(32, 32, 3), num_classes=NUM_CLASSES)
model.summary()

print(f'\n총 파라미터: {model.count_params():,}')

In [ ]:
# 모델 컴파일 및 학습

# 모델 저장 디렉토리 생성
os.makedirs('checkpoints', exist_ok=True)

# 모델 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백 정의
callbacks = [
    # EarlyStopping: 검증 손실이 개선되지 않으면 조기 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,               # 5 에폭 연속 개선 없으면 종료
        restore_best_weights=True, # 최적 가중치로 복원
        verbose=1
    ),
    
    # ModelCheckpoint: 최고 성능 모델 자동 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/cifar10_best.keras',
        monitor='val_accuracy',
        save_best_only=True,       # 최고 성능일 때만 저장
        verbose=1
    ),
    
    # ReduceLROnPlateau: 성능 정체 시 학습률 감소
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,                # 학습률 절반으로 감소
        patience=3,                # 3 에폭 개선 없으면 적용
        min_lr=1e-6,               # 최소 학습률
        verbose=1
    )
]

print('모델 컴파일 완료')
print(f'학습률: {model.optimizer.learning_rate.numpy():.0e}')
print(f'콜백: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau')
print()

# 모델 학습
history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.1,   # 학습 데이터의 10%를 검증용으로 사용
    callbacks=callbacks,
    verbose=1
)

print(f'\n학습 완료 (총 {len(history.history["loss"])} 에폭)')

In [ ]:
# 학습 곡선 시각화

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_ran = len(history.history['loss'])
epoch_range = range(1, epochs_ran + 1)

# 1. 손실 곡선
axes[0].plot(epoch_range, history.history['loss'],
             'b-o', markersize=4, label='학습 손실')
axes[0].plot(epoch_range, history.history['val_loss'],
             'r-o', markersize=4, label='검증 손실')
axes[0].set_xlabel('에폭')
axes[0].set_ylabel('손실 (Cross-Entropy)')
axes[0].set_title('학습/검증 손실 곡선')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. 정확도 곡선
axes[1].plot(epoch_range, history.history['accuracy'],
             'b-o', markersize=4, label='학습 정확도')
axes[1].plot(epoch_range, history.history['val_accuracy'],
             'r-o', markersize=4, label='검증 정확도')
axes[1].set_xlabel('에폭')
axes[1].set_ylabel('정확도')
axes[1].set_title('학습/검증 정확도 곡선')
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 3. 학습률 변화 (ReduceLROnPlateau 효과 확인)
if 'lr' in history.history:
    axes[2].semilogy(epoch_range, history.history['lr'],
                     'g-o', markersize=4)
    axes[2].set_xlabel('에폭')
    axes[2].set_ylabel('학습률 (로그 스케일)')
    axes[2].set_title('학습률 변화 (ReduceLROnPlateau)')
    axes[2].grid(True, alpha=0.3)
else:
    axes[2].text(0.5, 0.5, '학습률 기록 없음\n(콜백 미사용)',
                  ha='center', va='center', transform=axes[2].transAxes)

# 최종 성능 주석
final_train_acc = history.history['accuracy'][-1]
final_val_acc   = history.history['val_accuracy'][-1]
best_val_acc    = max(history.history['val_accuracy'])

plt.suptitle(
    f'CIFAR-10 CNN 학습 곡선\n'
    f'최종 학습 정확도: {final_train_acc:.4f} | '
    f'최종 검증 정확도: {final_val_acc:.4f} | '
    f'최고 검증 정확도: {best_val_acc:.4f}',
    fontsize=12
)
plt.tight_layout()
plt.show()

In [ ]:
# 테스트 세트 평가 및 혼동 행렬 시각화

# 테스트 세트 최종 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'테스트 손실: {test_loss:.4f}')
print(f'테스트 정확도: {test_acc:.4f} ({test_acc*100:.2f}%)')
print()

# 예측값 계산
y_pred_proba = model.predict(x_test, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# 분류 리포트
print('분류 보고서:')
print(classification_report(
    y_test, y_pred,
    target_names=CLASS_NAMES,
    digits=4
))

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred)

# 혼동 행렬 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 절대값 혼동 행렬
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=CLASS_NAMES,
    yticklabels=CLASS_NAMES,
    ax=axes[0]
)
axes[0].set_xlabel('예측 레이블')
axes[0].set_ylabel('실제 레이블')
axes[0].set_title('혼동 행렬 (절대값)')
axes[0].tick_params(axis='x', rotation=45)

# 정규화 혼동 행렬 (재현율 관점)
cm_normalized = cm.astype(float) / cm.sum(axis=1, keepdims=True)
sns.heatmap(
    cm_normalized, annot=True, fmt='.2f', cmap='YlOrRd',
    xticklabels=CLASS_NAMES,
    yticklabels=CLASS_NAMES,
    vmin=0, vmax=1,
    ax=axes[1]
)
axes[1].set_xlabel('예측 레이블')
axes[1].set_ylabel('실제 레이블')
axes[1].set_title('혼동 행렬 (행별 정규화 = 재현율)')
axes[1].tick_params(axis='x', rotation=45)

plt.suptitle(f'CIFAR-10 분류 결과 (테스트 정확도: {test_acc:.4f})', fontsize=13)
plt.tight_layout()
plt.show()

# 클래스별 정확도 분석
print('\n클래스별 정확도:')
class_acc = cm_normalized.diagonal()
for name, acc in sorted(zip(CLASS_NAMES, class_acc), key=lambda x: x[1], reverse=True):
    bar = '█' * int(acc * 20) + '░' * (20 - int(acc * 20))
    print(f'  {name:<8} {bar} {acc:.4f}')

In [ ]:
# 오분류 샘플 분석: 어떤 이미지를 잘못 분류했는지 확인

# 오분류된 인덱스 찾기
wrong_idx = np.where(y_pred != y_test)[0]
print(f'오분류된 샘플 수: {len(wrong_idx)} / {len(y_test)} '
      f'({len(wrong_idx)/len(y_test)*100:.1f}%)')

# 오분류 샘플 시각화 (15개)
n_show = min(15, len(wrong_idx))
selected = np.random.choice(wrong_idx, n_show, replace=False)

fig, axes = plt.subplots(3, 5, figsize=(16, 10))
for ax, idx in zip(axes.flatten(), selected):
    ax.imshow(x_test[idx])
    true_label = CLASS_NAMES[y_test[idx]]
    pred_label = CLASS_NAMES[y_pred[idx]]
    confidence = y_pred_proba[idx, y_pred[idx]]
    
    ax.set_title(
        f'실제: {true_label}\n예측: {pred_label} ({confidence:.2f})',
        fontsize=9,
        color='red'
    )
    ax.axis('off')

plt.suptitle('오분류 샘플 분석 (빨간 글씨: 오분류)', fontsize=13)
plt.tight_layout()
plt.show()

## 도전 과제

### 과제 1: 배치 정규화 제거 후 성능 비교
```python
# BatchNormalization 레이어를 제거한 모델 구현
def build_cnn_no_bn(input_shape=(32, 32, 3), num_classes=10):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                               input_shape=input_shape),
        # BatchNormalization 없이 MaxPooling으로 바로 이동
        tf.keras.layers.MaxPooling2D((2, 2)),
        # ... 동일한 구조로 계속
    ])
    return model
```
- BN 유/무의 학습 속도, 최종 정확도, 수렴 안정성을 비교하라

### 과제 2: Dropout 비율 변경
- `Dropout(0.0)`, `Dropout(0.3)`, `Dropout(0.5)`, `Dropout(0.7)` 비교
- 학습/검증 정확도 차이를 통해 과적합 정도를 분석하라

### 과제 3: 데이터 증강 추가
```python
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])
```
- 데이터 증강 유/무의 성능 차이를 비교하라

### 과제 4: Conv2D 채널 수 실험
- `[32, 64, 128]` vs `[64, 128, 256]` vs `[16, 32, 64]`
- 파라미터 수와 성능의 트레이드오프를 분석하라